## Data Preprocessing
1. Save id_map.json
2. Save {dataset_name}.inter

In [2]:
import pandas as pd

from utils.dataset import amazon_dataset2fullname

In [23]:
# 경로: {dataset_name}/
# Given files
"""
    reviews_{full_dataset_name}_5.json
    meta_{full_dataset_name}.json

"""

dataset_name = "clothing"
full_dataset_name = amazon_dataset2fullname[dataset_name]

reviews_path = f"{dataset_name}/reviews_{full_dataset_name}_5.json"

rating_df = pd.read_json(reviews_path, lines=True, encoding='utf-8')
   

## Save .inter

In [ ]:
# select columns
uirt_df = rating_df[['reviewerID', 'asin', 'overall', 'unixReviewTime']]

# str to float
uirt_df['overall'] = uirt_df['overall'].astype(float)
uirt_df['unixReviewTime'] = uirt_df['unixReviewTime'].astype(int)

# rename columns
uirt_df.columns = ['user_id:token', 'item_id:token', 'rating:float', 'timestamp:float']

# save to csv
uirt_df.to_csv(f"{dataset_name}/{dataset_name}.inter", index=False, sep='\t')

/tmp/ipykernel_1822050/701035166.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uirt_df['overall'] = uirt_df['overall'].astype(float)
/tmp/ipykernel_1822050/701035166.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  uirt_df['unixReviewTime'] = uirt_df['unixReviewTime'].astype(int)


## Save id_map.json

In [34]:
import json

# for reference
dataset_name = 'beauty'
id_map_path = f"{dataset_name}/id_map.json"
with open(id_map_path, 'r') as f:
    id_map = json.load(f)

id_map.keys()


dict_keys(['user2id', 'item2id', 'id2user', 'id2item', 'attribute2id', 'id2attribute', 'attributeid2num'])

In [35]:
rating_df.keys()

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [49]:
import numpy as np
import pandas as pd
import json

# 경로와 설정
dataset_name = 'home'

inter_path = f"{dataset_name}/{dataset_name}.inter"  # 변경
field_separator = "\t"
encoding = "utf-8"

# 1) 헤더 파싱해서 컬럼 이름만 추출 (user_id, item_id 등)
with open(inter_path, "r", encoding=encoding) as f:
    head = f.readline().rstrip("\n")

columns, usecols, dtypes = [], [], {}
for field_type in head.split(field_separator):
    field, ftype = field_type.split(":")
    columns.append(field)
    usecols.append(field_type)
    dtypes[field_type] = str  # TOKEN은 str로 읽음 (FLOAT이면 np.float64)

# 2) 필요한 컬럼만 로드
df = pd.read_csv(
    inter_path,
    delimiter=field_separator,
    usecols=usecols,
    dtype=dtypes,
    encoding=encoding,
    engine="python",
)
df.columns = columns

# 3) RecBole 스타일 factorize 함수(+1, [PAD]=0, id->token/token->id)
def recbole_factorize_token_series(series: pd.Series):
    ids, uniques = pd.factorize(series.values)  # 0..K-1
    new_ids = ids + 1                           # 1..K
    mp = np.array(["[PAD]"] + list(uniques))    # id -> token
    token2id = {t: i for i, t in enumerate(mp)} # token -> id
    return new_ids.astype(np.int64), mp, token2id

# 4) user_id, item_id에 적용 (컬럼명이 다르면 바꿔주세요)
user_ids, user_id_mp, user_token2id = recbole_factorize_token_series(df["user_id"])
item_ids, item_id_mp, item_token2id = recbole_factorize_token_series(df["item_id"])

# 5) 매핑 결과를 데이터프레임에 반영(선택)
df_mapped = df.copy()
df_mapped["user_id"] = user_ids
df_mapped["item_id"] = item_ids

user_token2id.pop('[PAD]')
item_token2id.pop('[PAD]')

# id to str in user_token2id
user_token2id = {k: str(v) for k, v in user_token2id.items()}
item_token2id = {k: str(v) for k, v in item_token2id.items()}

# user_token2id의 key, value 뒤집기
user_id2token = {v: k for k, v in user_token2id.items()}
item_id2token = {v: k for k, v in item_token2id.items()}

# 6) 필요시 매핑 테이블을 파일로 저장(선택)
save_json = True
if save_json:
    with open(f"{dataset_name}/id_map.json", "w", encoding="utf-8") as f:
        json.dump(
            {
                "user2id": user_token2id,
                "item2id": item_token2id,
                "id2user": user_id2token,
                "id2item": item_id2token,
            },
            f,
            ensure_ascii=False,
        )
